Pruebas en el modelo v6

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory
import cv2
import sys
%matplotlib inline
import matplotlib.pyplot as plt
from detectors.detector_factory import detector_factory

image_ext = ['jpg', 'jpeg', 'png', 'webp']
video_ext = ['mp4', 'mov', 'avi', 'mkv']
time_stats = ['tot', 'load', 'pre', 'net', 'dec', 'post', 'merge']

In [2]:
sys.argv = ['-f', 'ctdetv2', '--demo', '../images/yolo/', '--arch','dlav6_34',  '--load_model' ,'../exp/ctdetv2/coco_dlav6/model_best.pth'] #cmd = 'ctdet --exp_id coco_dla --batch_size 4 --lr 1.25e-4  --gpus 0 --num_workers 0'
opt = opts().parse()
print(opt.task)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\ctdetv2\default
ctdetv2


In [3]:
opt = opts().init()

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\ctdetv2\default
heads {'hm': 80, 'wh': 2, 'reg': 2}


In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
#opt.debug = max(opt.debug, 1)
opt.debug = 0
Detector = detector_factory[opt.task]
detector = Detector(opt)

Creating model...
loaded ../exp/ctdetv2/coco_dlav6/model_best.pth, epoch 220


In [5]:
detector.model

DLASeg(
  (base): DLA(
    (base_layer): Sequential(
      (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (re

In [6]:
with torch.no_grad():  
    params = detector.model.named_parameters()
    for nombre_parametro, parametro in params:
        if "fc_ss.3.W" in nombre_parametro:
            W = parametro.data
        if "fc_ss.3.temp" in nombre_parametro:
            temp = parametro

In [7]:
W

tensor([[ 0.0699,  0.1077, -0.2450,  ...,  0.2182, -0.0612, -0.3657],
        [ 0.0627,  0.1483,  0.1455,  ...,  0.1987, -0.3940, -0.2464],
        [ 0.1115,  0.5154,  0.2488,  ..., -0.0270,  0.0724, -0.3536],
        ...,
        [-0.0276,  0.2400, -0.1303,  ...,  0.1793,  0.0223,  0.2665],
        [ 0.1154,  0.1622,  0.0403,  ..., -0.1505,  0.1649, -0.3762],
        [-0.0689,  0.1931, -0.0773,  ..., -0.4931, -0.0268, -0.1302]],
       device='cuda:0')

In [8]:
temp

Parameter containing:
tensor(10.6730, device='cuda:0', requires_grad=True)

In [9]:
with torch.no_grad():  
    params = detector.model.named_parameters()
    for nombre_parametro, parametro in params:
        print(nombre_parametro)

base.base_layer.0.weight
base.base_layer.1.weight
base.base_layer.1.bias
base.level0.0.weight
base.level0.1.weight
base.level0.1.bias
base.level1.0.weight
base.level1.1.weight
base.level1.1.bias
base.level2.tree1.conv1.weight
base.level2.tree1.bn1.weight
base.level2.tree1.bn1.bias
base.level2.tree1.conv2.weight
base.level2.tree1.bn2.weight
base.level2.tree1.bn2.bias
base.level2.tree2.conv1.weight
base.level2.tree2.bn1.weight
base.level2.tree2.bn1.bias
base.level2.tree2.conv2.weight
base.level2.tree2.bn2.weight
base.level2.tree2.bn2.bias
base.level2.root.conv.weight
base.level2.root.bn.weight
base.level2.root.bn.bias
base.level2.project.0.weight
base.level2.project.1.weight
base.level2.project.1.bias
base.level3.tree1.tree1.conv1.weight
base.level3.tree1.tree1.bn1.weight
base.level3.tree1.tree1.bn1.bias
base.level3.tree1.tree1.conv2.weight
base.level3.tree1.tree1.bn2.weight
base.level3.tree1.tree1.bn2.bias
base.level3.tree1.tree2.conv1.weight
base.level3.tree1.tree2.bn1.weight
base.leve

In [10]:
W.shape 

torch.Size([256, 80])

In [11]:
class_name_coco = [
      '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
      'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
      'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
      'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
      'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
      'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
      'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
      'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
      'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
      'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
      'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
      'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
      'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [12]:
opt.demo = '../images/DBPaper/'
image_names = [opt.demo]
images_path = {}
ls = os.listdir(opt.demo)

for folder_name in sorted(ls):
    image_names = []
    ls2 = os.listdir(os.path.join(opt.demo, folder_name))
    for file_name in sorted(ls2):
      ext = file_name[file_name.rfind('.') + 1:].lower()
      if ext in image_ext:
          image_names.append(os.path.join(opt.demo, folder_name, file_name).replace("\\","/"))
    images_path[folder_name] = image_names
"""
images_path['ak47'] :
 ['../images/256_ObjectCategories/Caltech80/train/ak47/001_0002.jpg',
  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0003.jpg',
  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0029.jpg',
  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0036.jpg',
  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0058.jpg'],
"""

"\nimages_path['ak47'] :\n ['../images/256_ObjectCategories/Caltech80/train/ak47/001_0002.jpg',\n  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0003.jpg',\n  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0029.jpg',\n  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0036.jpg',\n  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0058.jpg'],\n"

In [13]:
class_name_yolo = list(images_path.keys())

In [14]:
for index_w, folder in enumerate(class_name_yolo): 
    print(folder)
    for (image_name) in images_path[folder]:
        print(image_name)

aceite
../images/DBPaper/aceite/aceite_0.png
../images/DBPaper/aceite/aceite_1.png
../images/DBPaper/aceite/aceite_10.png
../images/DBPaper/aceite/aceite_11.png
../images/DBPaper/aceite/aceite_2.png
../images/DBPaper/aceite/aceite_3.png
../images/DBPaper/aceite/aceite_4.png
../images/DBPaper/aceite/aceite_5.png
../images/DBPaper/aceite/aceite_6.png
../images/DBPaper/aceite/aceite_7.png
../images/DBPaper/aceite/aceite_8.png
../images/DBPaper/aceite/aceite_9.png
aerosol
../images/DBPaper/aerosol/aerosol_0.png
../images/DBPaper/aerosol/aerosol_1.png
../images/DBPaper/aerosol/aerosol_10.png
../images/DBPaper/aerosol/aerosol_11.png
../images/DBPaper/aerosol/aerosol_2.png
../images/DBPaper/aerosol/aerosol_3.png
../images/DBPaper/aerosol/aerosol_4.png
../images/DBPaper/aerosol/aerosol_5.png
../images/DBPaper/aerosol/aerosol_6.png
../images/DBPaper/aerosol/aerosol_7.png
../images/DBPaper/aerosol/aerosol_8.png
../images/DBPaper/aerosol/aerosol_9.png
azucarero
../images/DBPaper/azucarero/azucare

In [15]:
W_b = torch.zeros((256,40))

for index_w, folder in enumerate(images_path.keys()): #index_w [0->39]
    imagenes_max = 5
    for (image_name) in images_path[folder]:        
        
        if not(imagenes_max):
            break
        imagenes_max -= 1        
        
        ret = detector.run(image_name)
        output = ret['output']
        pred_obj = output['objns']
        hm = pred_obj.squeeze(0).squeeze(0)
        scores = output['hm'].squeeze(0)
        batch, cat, height, width = pred_obj.size()
        topk_scores, topk_inds = torch.topk(pred_obj.view(batch, cat, -1), 1)
        

        topk_inds = topk_inds % (height * width)
        topk_ys   = (topk_inds / width).int()
        topk_xs   = (topk_inds % width).int()
        
        radio = 3
        #eje_x, eje_y = int(topk_xs), int(topk_ys)
        eje_x, eje_y = 62,64

        descriptores = output['descriptor'].squeeze(0)[:,eje_x,eje_y]
        count = 1
        for x in range(eje_x - radio, eje_x + radio+1):
            for y in range(eje_y - radio, eje_y + radio+1):
                if not (x==eje_x and y==eje_y):
                    descriptores = (descriptores+output['descriptor'].squeeze(0)[:,x,y])
                    count += 1
        W_b[:,index_w] = descriptores/count
        print(image_name)
        print('-------------------------')

../images/DBPaper/aceite/aceite_0.png
-------------------------
../images/DBPaper/aceite/aceite_1.png
-------------------------
../images/DBPaper/aceite/aceite_10.png
-------------------------
../images/DBPaper/aceite/aceite_11.png
-------------------------
../images/DBPaper/aceite/aceite_2.png
-------------------------
../images/DBPaper/aerosol/aerosol_0.png
-------------------------
../images/DBPaper/aerosol/aerosol_1.png
-------------------------
../images/DBPaper/aerosol/aerosol_10.png
-------------------------
../images/DBPaper/aerosol/aerosol_11.png
-------------------------
../images/DBPaper/aerosol/aerosol_2.png
-------------------------
../images/DBPaper/azucarero/azucarero_0.png
-------------------------
../images/DBPaper/azucarero/azucarero_1.png
-------------------------
../images/DBPaper/azucarero/azucarero_10.png
-------------------------
../images/DBPaper/azucarero/azucarero_11.png
-------------------------
../images/DBPaper/azucarero/azucarero_2.png
--------------------

../images/DBPaper/leche/leche_11.png
-------------------------
../images/DBPaper/leche/leche_2.png
-------------------------
../images/DBPaper/limon/limon_0.png
-------------------------
../images/DBPaper/limon/limon_1.png
-------------------------
../images/DBPaper/limon/limon_10.png
-------------------------
../images/DBPaper/limon/limon_11.png
-------------------------
../images/DBPaper/limon/limon_2.png
-------------------------
../images/DBPaper/linterna/linterna_0.png
-------------------------
../images/DBPaper/linterna/linterna_1.png
-------------------------
../images/DBPaper/linterna/linterna_10.png
-------------------------
../images/DBPaper/linterna/linterna_11.png
-------------------------
../images/DBPaper/linterna/linterna_2.png
-------------------------
../images/DBPaper/mantequilla/mantequilla_0.png
-------------------------
../images/DBPaper/mantequilla/mantequilla_1.png
-------------------------
../images/DBPaper/mantequilla/mantequilla_10.png
------------------------

In [16]:
W_b = W_b/temp
print(W_b)

tensor([[-0.7145, -0.7577, -0.1851,  ...,  0.1388, -0.1531, -0.1136],
        [ 0.1401, -0.1771, -0.1194,  ...,  0.1220, -0.0365,  0.0991],
        [-0.5538, -0.1455, -0.3468,  ..., -0.2506, -0.5569, -0.6188],
        ...,
        [ 0.5249,  0.3601, -0.3793,  ..., -0.0466,  0.3697,  0.3055],
        [-0.2527, -0.1101,  0.0236,  ..., -0.2854, -0.3428,  0.0705],
        [-0.1696, -0.0350,  0.1151,  ...,  0.2480,  0.3776,  0.0850]],
       grad_fn=<DivBackward0>)


In [17]:
W_b = torch.cat([W_b,W_b], dim=1)
print(W_b.size())

torch.Size([256, 80])


In [18]:
with torch.no_grad():  
    params = detector.model.named_parameters()
    for nombre_parametro, parametro in params:
        if "fc_ss.3.W" in nombre_parametro:
            parametro.data = W_b

In [19]:
with torch.no_grad():  
    params = detector.model.named_parameters()
    for nombre_parametro, parametro in params:
        if "fc_ss.3.W" in nombre_parametro:
            print(parametro.data)

tensor([[-0.7145, -0.7577, -0.1851,  ...,  0.1388, -0.1531, -0.1136],
        [ 0.1401, -0.1771, -0.1194,  ...,  0.1220, -0.0365,  0.0991],
        [-0.5538, -0.1455, -0.3468,  ..., -0.2506, -0.5569, -0.6188],
        ...,
        [ 0.5249,  0.3601, -0.3793,  ..., -0.0466,  0.3697,  0.3055],
        [-0.2527, -0.1101,  0.0236,  ..., -0.2854, -0.3428,  0.0705],
        [-0.1696, -0.0350,  0.1151,  ...,  0.2480,  0.3776,  0.0850]])


In [20]:
torch.save(detector.model.state_dict(), "model_wb_yolo.pth")